In [1]:
import pandas as pd
import datetime
import matplotlib.pyplot as plt

In [2]:
import warnings
warnings.filterwarnings("ignore")

# Load data

In [3]:
save_address = 'C:/Users/xi-lucy.chen/Documents/GitLab/data_challenge_alc_total/phase_2_Lucy/feature/'
wp1 = pd.read_csv(save_address+'wp1.csv')
wp2 = pd.read_csv(save_address+'wp2.csv')
wp3 = pd.read_csv(save_address+'wp3.csv')
wp4 = pd.read_csv(save_address+'wp4.csv')
wp5 = pd.read_csv(save_address+'wp5.csv')
wp6 = pd.read_csv(save_address+'wp6.csv')


In [4]:
for wp in [wp1,wp2,wp3,wp4,wp5,wp6]:
    wp.index = wp['date']
    wp.drop(columns='date',inplace=True)

In [5]:
wp1[0:2]

,wp1,train,u,v,ws,wd,Day sin,Day cos,Year sin,Year cos,...,dff_v_mean,dff_ws_mean,dff_u_1%,dff_v_1%,dff_ws_1%,dff_wd_1%,dff_u_-1%,dff_v_-1%,dff_ws_-1%,dff_wd_-1%
date,,,,,,,,,,,,,,,,,,,,,
2009-07-01 00:00:00,0.051,1.0,3.14,-3.62,4.79,139.09,0.866025,0.500000,0.015690,-0.999877,...,-3.158256,-0.795149,NaN,NaN,NaN,0.000000,0.000000,-0.000000,0.000000,0.000000
2009-07-01 01:00:00,0.051,1.0,3.14,-3.62,4.79,139.09,0.965926,0.258819,0.014973,-0.999888,...,-3.158250,-0.795237,0.0,-0.0,0.0,-0.008412,0.057325,0.016575,0.033403,-0.008412


In [6]:
def get_X(df):
    return df.iloc[:, 2:]

def get_y(df, name='wp1'):
    return df[name]

def get_X_test(df):
    return df.iloc[:, 1:]


def data_test_split(df, model_address, start_date="2009-07-01 00:00:00", end_date="2010-12-31 23:00:00", name='wp1'):
    '''
    resplit data & test from full data
    '''
    train = df[df['train'] == 1].sort_values(by='date')
    test = df[df['train'] != 1].sort_values(by='date')
    train = train.drop(columns=['train'])
    test = test.drop(columns=['train', name])
    train = train.loc[start_date:end_date]
    train.to_csv(model_address+'training_data_{}.csv'.format(name))
    X_train = get_X(train)
    y_train = get_y(train, name)
    X_forecast = get_X_test(test)
    return X_train, y_train, X_forecast ,test # train, test,


# Data Split

In [7]:
from sklearn.model_selection import train_test_split


def split_data(X, y):
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=177)
    return X_train, X_test, y_train, y_test


# XGBOOST

In [8]:
import model1

In [9]:
tune_parameter = False
grid_search = False
params = {'colsample_bytree': 0.6,
          'max_depth': 13,
          'min_child_weight': 9,
          'eval_metric': 'mae',
          'subsample': 0.6,
          'colsample': 1.0,
          'eta': 0.05}

num_boost_round = 1000

# FORECAST

In [10]:
def prediction(X_forecast, test, model, name):
    X_forecast = xgb.DMatrix(data=X_forecast)
    df_predictions = pd.DataFrame({
        'date': test.index,
        name: model.predict(X_forecast),
    })
    save_address = 'C:/Users/xi-lucy.chen/Documents/GitLab/data_challenge_alc_total/phase 2 model/Lucy/result/'
    df_predictions.to_csv(save_address+"pred_{}.csv".format(name))
    return df_predictions


# RUNNNNNN

In [53]:
#from pandas.io.json import json_normalize
from datetime import date
import json
import gc
#param_list = pd.DataFrame()
prediction = pd.DataFrame()
start_date = "2009-07-01 00:00:00"
end_date = "2010-12-31 23:00:00"

save_address = 'C:/Users/xi-lucy.chen/Documents/GitLab/data_challenge_alc_total/phase_2_Lucy/result/'
model_address = 'C:/Users/xi-lucy.chen/Documents/GitLab/data_challenge_alc_total/phase_2_Lucy/model/'
months_predict = ['201101', '201102', '201103','201104','201105','201106','201107','201108','201109','201110','201111','201112','201201','201202','201203','201204','201205','201206']
train_end_date = ["2010-12-31 23:00:00",
                "2011-01-31 23:00:00",
                "2011-02-28 23:00:00",
                "2011-03-31 23:00:00",
                "2011-04-30 23:00:00",
                "2011-05-30 23:00:00",
                "2011-06-30 23:00:00",
                "2011-07-31 23:00:00",
                "2011-08-31 23:00:00",
                "2011-09-30 23:00:00",
                "2011-10-31 23:00:00",  
                "2011-11-30 23:00:00",
                "2011-12-31 23:00:00",
                "2012-01-31 23:00:00",
                "2012-02-29 23:00:00",
                "2012-03-31 23:00:00",
                "2012-04-30 23:00:00",
                "2012-05-31 23:00:00"]

for year, end_date in zip(months_predict, train_end_date):
    # for name,df in zip(["wp1", "wp2", "wp3", "wp4", "wp5", "wp6"],[wp1,wp2,wp3,wp4,wp5,wp6]):
    #     print('------ name is :', name, "------")
    #     #split my merged dataset
    #     X, y, X_forecast,test = data_test_split(df, model_address=model_address, start_date=start_date, end_date=end_date, name=name)
    #     #split to train model
    #     '''we do the modification here'''
    for name_1, df_1 in zip(["wp1", "wp2", "wp3", "wp4", "wp5", "wp6"], [wp1, wp2, wp3, wp4, wp5, wp6]):
            print('I am merging ..',name_1)
            X_temp, y_temp, X_forecast_temp,test_temp = data_test_split(
                df_1, model_address=model_address, start_date=start_date, end_date=end_date, name=name_1)
            test_temp['wp'] = name_1
            if name_1 == 'wp1':
                my_X = X_temp
                my_y = y_temp
                my_X_forecast = X_forecast_temp
                my_test =test_temp
                print('my X length is..', len(my_X))
                print('my y length is..', len(my_y))
            else:
                my_X = my_X.append(X_temp)
                my_y = my_y.append(y_temp)
                my_X_forecast = my_X_forecast.append(X_forecast_temp)
                my_test = my_test.append(test_temp)
                print('my X length is..', len(my_X))
                print('my y length is..', len(my_y))
                print('my_X_forecast length is..', len(my_X_forecast))
                print('my_test length is..', len(my_test))
                del X_temp, y_temp, X_forecast_temp, test_temp
                gc.collect()

    X = my_X
    y = my_y
    X_forecast = my_X_forecast
    test=my_test

    ''''''
    X_train, X_test, y_train, y_test = split_data(X,y)
    model,param = model1.run_xgb(X_train, X_test, y_train, y_test, params, num_boost_round=num_boost_round,
                            tune_parameter=False, grid_search=False, graph=False)
    '''
    Forecast
    '''
    X_forecast = xgb.DMatrix(data=X_forecast)
    df_predictions = pd.DataFrame({
            'date': test.index,
            'wp':test['wp'],
            "forecast": model.predict(X_forecast),
        })
    model.save_model(model_address+"model_merged_train.json")
        #df_predictions.to_csv(save_address+"pred_{}.csv".format(name))
        #print(param)
        #print(type(param)) #This is string type
        #param = json.loads(param)
    with open(model_address+"param_merged_train.json", 'w') as f:
            json.dump(param, f)
        #param_list = param_list.append(param)
    # if name =='wp1':
    #         prediction = df_predictions
    #     else:
    #         prediction = prediction.merge(df_predictions,on=['date'])
    prediction= pd.pivot_table(df_predictions, index=df_predictions.index,
                   columns='wp', values='forecast', aggfunc=min).reset_index()

    prediction.rename(columns={'date': 'date1'}, inplace=True)
    data_path = 'C:/Users/xi-lucy.chen/Documents/GitLab/data_challenge_alc_total/data/'
    test = pd.read_csv(f'{data_path}test.csv')
    prediction = pd.concat([test, prediction], axis=1)
    prediction.drop(columns='date1', inplace=True)

    save_address = 'C:/Users/xi-lucy.chen/Documents/GitLab/data_challenge_alc_total/phase_2_Lucy/result/before_submission/'
    prediction.to_csv(
    save_address+'predictions_end_date_{}.csv'.format(year), index=False, sep=';')


I am merging .. wp1
my X length is.. 13176
my y length is.. 13176
I am merging .. wp2
my X length is.. 26352
my y length is.. 26352
my_X_forecast length is.. 14880
my_test length is.. 14880
I am merging .. wp3
my X length is.. 39528
my y length is.. 39528
my_X_forecast length is.. 22320
my_test length is.. 22320
I am merging .. wp4
my X length is.. 52704
my y length is.. 52704
my_X_forecast length is.. 29760
my_test length is.. 29760
I am merging .. wp5
my X length is.. 65880
my y length is.. 65880
my_X_forecast length is.. 37200
my_test length is.. 37200
I am merging .. wp6
my X length is.. 79056
my y length is.. 79056
my_X_forecast length is.. 44640
my_test length is.. 44640
Baseline MAE is 0.23
[14:13:35] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { colsample } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are n

<IPython.core.display.Javascript object>

I am merging .. wp1
my X length is.. 13488
my y length is.. 13488
I am merging .. wp2
my X length is.. 26976
my y length is.. 26976
my_X_forecast length is.. 14880
my_test length is.. 14880
I am merging .. wp3
my X length is.. 40464
my y length is.. 40464
my_X_forecast length is.. 22320
my_test length is.. 22320
I am merging .. wp4
my X length is.. 53952
my y length is.. 53952
my_X_forecast length is.. 29760
my_test length is.. 29760
I am merging .. wp5
my X length is.. 67440
my y length is.. 67440
my_X_forecast length is.. 37200
my_test length is.. 37200
I am merging .. wp6
my X length is.. 80928
my y length is.. 80928
my_X_forecast length is.. 44640
my_test length is.. 44640
Baseline MAE is 0.23
[14:13:53] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { colsample } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are n

<IPython.core.display.Javascript object>

I am merging .. wp1
my X length is.. 13776
my y length is.. 13776
I am merging .. wp2
my X length is.. 27552
my y length is.. 27552
my_X_forecast length is.. 14880
my_test length is.. 14880
I am merging .. wp3
my X length is.. 41328
my y length is.. 41328
my_X_forecast length is.. 22320
my_test length is.. 22320
I am merging .. wp4
my X length is.. 55104
my y length is.. 55104
my_X_forecast length is.. 29760
my_test length is.. 29760
I am merging .. wp5
my X length is.. 68880
my y length is.. 68880
my_X_forecast length is.. 37200
my_test length is.. 37200
I am merging .. wp6
my X length is.. 82656
my y length is.. 82656
my_X_forecast length is.. 44640
my_test length is.. 44640
Baseline MAE is 0.24
[14:14:15] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { colsample } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are n

...

I am merging .. wp1
my X length is.. 14401
my y length is.. 14401
I am merging .. wp2
my X length is.. 28802
my y length is.. 28802
my_X_forecast length is.. 14880
my_test length is.. 14880
I am merging .. wp3
my X length is.. 43203
my y length is.. 43203
my_X_forecast length is.. 22320
my_test length is.. 22320
I am merging .. wp4
my X length is.. 57604
my y length is.. 57604
my_X_forecast length is.. 29760
my_test length is.. 29760
I am merging .. wp5
my X length is.. 72005
my y length is.. 72005
my_X_forecast length is.. 37200
my_test length is.. 37200
I am merging .. wp6
my X length is.. 86406
my y length is.. 86406
my_X_forecast length is.. 44640
my_test length is.. 44640
Baseline MAE is 0.24
[14:14:58] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { colsample } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are n

<IPython.core.display.Javascript object>

I am merging .. wp1
my X length is.. 14712
my y length is.. 14712
I am merging .. wp2
my X length is.. 29424
my y length is.. 29424
my_X_forecast length is.. 14880
my_test length is.. 14880
I am merging .. wp3
my X length is.. 44136
my y length is.. 44136
my_X_forecast length is.. 22320
my_test length is.. 22320
I am merging .. wp4
my X length is.. 58848
my y length is.. 58848
my_X_forecast length is.. 29760
my_test length is.. 29760
I am merging .. wp5
my X length is.. 73560
my y length is.. 73560
my_X_forecast length is.. 37200
my_test length is.. 37200
I am merging .. wp6
my X length is.. 88272
my y length is.. 88272
my_X_forecast length is.. 44640
my_test length is.. 44640
Baseline MAE is 0.24
[14:15:21] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { colsample } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are n

<IPython.core.display.Javascript object>

I am merging .. wp1
my X length is.. 15024
my y length is.. 15024
I am merging .. wp2
my X length is.. 30048
my y length is.. 30048
my_X_forecast length is.. 14880
my_test length is.. 14880
I am merging .. wp3
my X length is.. 45072
my y length is.. 45072
my_X_forecast length is.. 22320
my_test length is.. 22320
I am merging .. wp4
my X length is.. 60096
my y length is.. 60096
my_X_forecast length is.. 29760
my_test length is.. 29760
I am merging .. wp5
my X length is.. 75120
my y length is.. 75120
my_X_forecast length is.. 37200
my_test length is.. 37200
I am merging .. wp6
my X length is.. 90144
my y length is.. 90144
my_X_forecast length is.. 44640
my_test length is.. 44640
Baseline MAE is 0.24
[14:15:47] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { colsample } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are n

In [ ]:
def read_folder(file_folder):
    """
        Read all files in the folder
        """
    files = os.listdir(file_folder)
    df = []
    for f in files:
        print(f)
        my_file = file_folder + "/" + f
        temp = pd.read_csv(my_file, sep=';')
        temp['file_name']=f
        df.append(temp)
    df_full = pd.concat(df, ignore_index=True)
    return df_full


In [ ]:
df_full = read_folder(
    "C:/Users/xi-lucy.chen/Documents/GitLab/data_challenge_alc_total/phase_2_Lucy/result/before_submission/")

In [ ]:
df_full['file_forecast_month'] = df_full['file_name'].str[21:27]
df_full['forecast_month'] = df_full['date'].astype(str).str[0:6]

df_full['match'] = np.where(df_full['file_forecast_month']
                       == df_full['forecast_month'],True,False)
df_full=df_full[df_full['match']==True]
prediction = df_full.drop(
    columns=['file_forecast_month', 'match', 'file_name', 'forecast_month'])
save_address2 = 'C:/Users/xi-lucy.chen/Documents/GitLab/data_challenge_alc_total/phase_2_Lucy/result/submission/'
prediction.to_csv(
    save_address2+'predictions_{}.csv'.format(date.today()), index=False, sep=';')


# Merge predictions

read_address = 'C:/Users/xi-lucy.chen/Documents/GitLab/data_challenge_alc_total/phase 2 model/Lucy/result/'
pred_wp1=pd.read_csv(read_address+"pred_wp1.csv")
pred_wp2=pd.read_csv(read_address+"pred_wp2.csv")
pred_wp3=pd.read_csv(read_address+"pred_wp3.csv")
pred_wp4=pd.read_csv(read_address+"pred_wp4.csv")
pred_wp5=pd.read_csv(read_address+"pred_wp5.csv")
pred_wp6=pd.read_csv(read_address+"pred_wp6.csv")

In [ ]:
# read_address = 'C:/Users/xi-lucy.chen/Documents/GitLab/data_challenge_alc_total/phase 2 model/Lucy/result/'
# save_address = 'C:/Users/xi-lucy.chen/Documents/GitLab/data_challenge_alc_total/phase 2 model/Lucy/result/submission/'
# pred1=pd.read_csv(read_address+'pred_wp1.csv')
# pred2=pd.read_csv(read_address+'pred_wp2.csv')
# pred3=pd.read_csv(read_address+'pred_wp3.csv')
# pred4=pd.read_csv(read_address+'pred_wp4.csv')
# pred5=pd.read_csv(read_address+'pred_wp5.csv')
# pred6 = pd.read_csv(read_address+'pred_wp6.csv')

# merge_pred=pd.DataFrame()
# merge_pred=pred1
# merge_pred = merge_pred.merge(pred2,on='date')
# merge_pred = merge_pred.merge(pred3,on='date')
# merge_pred = merge_pred.merge(pred4,on='date')
# merge_pred = merge_pred.merge(pred5,on='date')
# merge_pred = merge_pred.merge(pred6,on='date')


# merge_pred.to_csv('predictions.csv', index=False, sep=';')
# df_predictions.head()


In [ ]:
import json
model_address = 'C:/Users/xi-lucy.chen/Documents/GitLab/data_challenge_alc_total/phase_2_Lucy/model/'
def read_my_json(model_address,my_file):
    f = open(model_address+my_file)
    json_file = json.load(f)
    return json_file


param1 = json.loads(read_my_json(model_address,'param_wp1.json'))
param2 = json.loads(read_my_json(model_address, 'param_wp2.json'))
param3=json.loads(read_my_json(model_address, 'param_wp3.json'))
param4=json.loads(read_my_json(model_address, 'param_wp4.json'))
param5=json.loads(read_my_json(model_address, 'param_wp5.json'))
param6=json.loads(read_my_json(model_address, 'param_wp6.json'))


In [ ]:
df = pd.json_normalize(param1)
df=df.append(pd.json_normalize(param2))
df=df.append(pd.json_normalize(param3))
df=df.append(pd.json_normalize(param4))
df=df.append(pd.json_normalize(param5))
df = df.append(pd.json_normalize(param6))
df=df.reset_index(drop=True)
df.to_csv(model_address + 'my_tuned_params.csv')
